## Data
pull down some data


In [3]:
import sys
sys.path
sys.path.append('buml') 
import os
import Data.utils
os.environ["DATASETSPATH"]=""
data_source = "red_wine.hdf5"
training = "/folds/1/training/(1|2|3|4|5|6|7|8)"
dataset_file = os.path.join(os.environ["DATASETSPATH"], data_source)
training_dataset = Data.BigDataset(dataset_file, training, "data")



 BIG DATASET Working on red_wine.hdf5
Entries /folds/1/training/(1|2|3|4|5|6|7|8)


In [7]:
a=training_dataset.sample_data(2)

In [10]:
a

(array([[ 8.    ,  0.42  ,  0.17  ,  2.    ,  0.073 ,  6.    , 18.    ,
          0.9972,  3.29  ,  0.61  ,  9.2   ],
        [10.4   ,  0.24  ,  0.49  ,  1.8   ,  0.075 ,  6.    , 20.    ,
          0.9977,  3.18  ,  1.06  , 11.    ]], dtype=float32),)

In [12]:
training_dataset.entries_regexp

'/folds/1/training/(1|2|3|4|5|6|7|8)'

In [13]:
len(training_dataset.element_names)

1

In [15]:
import numpy as np
file_index = np.random.randint(0, training_dataset.get_n_files())

In [17]:
training_dataset.get_n_files()

8

In [16]:
file_index

5

In [19]:
i=0
file_index=2
index=2
training_dataset.get_file(i, file_index)[ index:index + training_dataset.block_lengths[i], :].flatten()

array([ 6.3   ,  0.47  ,  0.    ,  1.4   ,  0.055 , 27.    , 33.    ,
        0.9922,  3.45  ,  0.48  , 12.3   ], dtype=float32)

In [22]:
training_dataset.get_file(i, file_index)

array([[ 7.9  ,  0.31 ,  0.32 , ...,  3.41 ,  0.56 , 12.6  ],
       [11.2  ,  0.4  ,  0.5  , ...,  3.1  ,  0.58 , 10.4  ],
       [ 6.3  ,  0.47 ,  0.   , ...,  3.45 ,  0.48 , 12.3  ],
       ...,
       [ 8.3  ,  0.26 ,  0.37 , ...,  3.26 ,  0.7  ,  9.6  ],
       [ 7.5  ,  0.685,  0.07 , ...,  3.38 ,  0.55 , 10.9  ],
       [ 6.4  ,  0.64 ,  0.21 , ...,  3.59 ,  0.66 ,  9.8  ]],
      dtype=float32)

In [1]:
import sys
sys.path.append("../deepnade/buml")
sys.path.append("../deepnade")

import os
from optparse import OptionParser
import Instrumentation
import Backends
import Optimization
import TrainingController
import numpy as np
import Utils
import Data.utils
import scipy.stats
import gc
from Utils.DropoutMask import create_dropout_masks
from Utils.theano_helpers import floatX

import h5py

import NADE

from orderlessNADE import *

In [3]:
parser = get_parser()

os.environ["RESULTSPATH"] = "/Users/marina/Documents/PhD/research/astro_research/code/NADE/output/"
os.environ["DATASETSPATH"] = "/Users/marina/Documents/PhD/research/astro_research/code/NADE/deepnade/"


In [4]:
options, args =parser.parse_args(["--theano", 
                                  "--form", "MoG", 
                                  "--dataset", "red_wine.hdf5", 
                                  "--training_route", "/folds/1/training/(1|2|3|4|5|6|7|8)",
                                 "--validation_route", "/folds/1/training/9",
                                 "--test_route", "/folds/1/tests/.*",
                                 "--samples_name", "data",
                                 "--hlayers", "2", # 2 hidden layers
                                  "--layerwise",
                                  "--lr", "0.02",
                                  "--wd", "0.02",
                                  "--n_components", "10",
                                  "--epoch_size", "100",
                                  "--momentum", "0.9",
                                  "--units", "100",
                                  "--pretraining_epochs", "5",
                                  "--validation_loops", "20",
                                  "--epochs", "20",
                                  "--normalize",
                                  "--batch_size", "100",
                                  "--show_training_stop", "red_wine"])



In [5]:
if options.theano:
    import NADE
else:
    import npNADE as NADE
    raise Exception("Not implemented yet")

results_route = os.path.join(os.environ["RESULTSPATH"], args[0])
try:
    os.makedirs(results_route)
except OSError:
    pass

console = Backends.Console()
textfile_log = Backends.TextFile(os.path.join(results_route, "NADE_training.log"))
hdf5_backend = Backends.HDF5(results_route, "NADE")
hdf5_backend.write([], "options", options)
hdf5_backend.write([], "svn_revision", Utils.svn.svnversion())
hdf5_backend.write([], "svn_status", Utils.svn.svnstatus())
hdf5_backend.write([], "svn_diff", Utils.svn.svndiff())

In [6]:
training_dataset, validation_dataset, test_dataset, masks_dataset = prep_datasets(options, hdf5_backend)



 BIG DATASET Working on /Users/marina/Documents/PhD/research/astro_research/code/NADE/deepnade/red_wine.hdf5
Entries /folds/1/training/(1|2|3|4|5|6|7|8)
New entries [<HDF5 group "/" (2 members)>]
Pats ['folds', '1', 'training', '(1|2|3|4|5|6|7|8)']
FINAL entries length: 8


 BIG DATASET Working on /Users/marina/Documents/PhD/research/astro_research/code/NADE/deepnade/red_wine.hdf5
Entries /folds/1/training/9
New entries [<HDF5 group "/" (2 members)>]
Pats ['folds', '1', 'training', '9']
FINAL entries length: 1


 BIG DATASET Working on /Users/marina/Documents/PhD/research/astro_research/code/NADE/deepnade/red_wine.hdf5
Entries /folds/1/tests/.*
New entries [<HDF5 group "/" (2 members)>]
Pats ['folds', '1', 'tests', '.*']
FINAL entries length: 1


 BIG DATASET Working on /var/folders/wv/_wh4dxyj3ng73fr13vk_n3700000gn/T/tmp1R3gLI
Entries /folds/1/training/(1|2|3|4|5|6|7|8)
New entries [<HDF5 group "/" (1 members)>]
Pats ['folds', '1', 'training', '(1|2|3|4|5|6|7|8)']
FINAL entries leng

../deepnade/buml/Data/BigDataset.py:103: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return np.array(self.files[element][index].value)  # , order='C')


In [6]:
n_visible = training_dataset.get_dimensionality(0)
l = 1 if options.layerwise else options.hlayers
l

1

In [7]:
nade_class, nade, loss_function, validation_loss_measurement = set_loss(
    options, l, n_visible, validation_dataset, masks_dataset)